In [5]:
import json
import os

import chromadb

import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

# Accepted file formats for that can be stored in
# a vector database instance
from autogen.retrieve_utils import TEXT_FORMATS
# When using a single openai endpoint, you can use the following:
config_list = [{"model": "gpt-4", "api_key": os.getenv("OPENAI_API_KEY")}]

In [2]:
import requests
from io import StringIO
from bs4 import BeautifulSoup
import os
def get_hed_vocab():
    if os.path.exists('HEDLatest_terms'):
        with open('HEDLatest_terms', 'r') as fin:
            return fin.read()
    else:
        # URL of the XML file
        url = "https://raw.githubusercontent.com/hed-standard/hed-schemas/main/standard_schema/hedxml/HEDLatest.xml"
        
        # Send a GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the XML content
            xml_content = response.text
            soup = BeautifulSoup(xml_content, "lxml")
        
            # Find all nodes and extract their names
            all_nodes = soup.find_all('node')
            node_names = [node.find('name', recursive=False).string for node in all_nodes]
            description = []
            for node in all_nodes:
                if node.find('description', recursive=False):
                    description.append(node.find('description', recursive=False).string)
                else:
                    description.append('')
                # print(node.find('description', recursive=False).string)
        
            return dict(zip(node_names, description))
            # return node_names
        else:
            print(f"Failed to retrieve data from the URL. Status code: {response.status_code}") 
hed_vocab_description = get_hed_vocab()

/var/folders/c2/k7vlgt7j1y306mj9yk_51jvr0000gn/T/ipykernel_49681/1805897931.py:20: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(xml_content, "lxml")


In [8]:
import json
with open('hed_vocab_description.json', 'w', encoding='utf-8') as out:
    json.dump(hed_vocab_description, out, indent=4)

In [12]:
# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list,
    },
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    # max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": [
            "https://raw.githubusercontent.com/hed-standard/hed-schemas/main/standard_schema/hedxml/HEDLatest.xml"
        ],
        # "custom_text_types": ["non-existent-type"],
        # "chunk_token_size": 2000,
        "model": config_list[0]["model"],
        # "client": chromadb.PersistentClient(path="/tmp/chromadb"),  # deprecated, use "vector_db" instead
        "vector_db": "chroma",  # to use the deprecated `client` parameter, set to None and uncomment the line above
        "overwrite": True,  # set to True if you want to overwrite an existing collection
    },
    code_execution_config=False,  # set to False if you don't want to execute the code
)

In [13]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# given a problem, we use the ragproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the ragproxyagent for processing.
# The conversation continues until the termination condition is met, in RetrieveChat, the termination condition when no human-in-loop is no code block detected.
# With human-in-loop, the conversation will continue until the user says "exit".
code_problem = "You have knowledge of the tags in the Hierarchical Event Descriptor (HED) system.\n" \
    "Tell me the tags that might be relevant to the following sentence description and why\n" \
    "A picture of a blue square on a red background\n" 
chat_result = ragproxyagent.initiate_chat(
    assistant, 
    message=ragproxyagent.message_generator, 
    problem=code_problem, 
)

Trying to create collection.


max_tokens is too small to fit a single line of text. Breaking this line:
	 ...
Failed to split docs with must_break_at_empty_line being True, set to False.
2024-06-11 14:08:51,356 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 15 chunks.
2024-06-11 14:08:51,361 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.
Number of requested results 20 is greater than number of elements in index 15, updating n_results = 15


VectorDB returns doc_ids:  [['a5042372', '8fc3eac8', '916dc60e', 'f69230a5', '36409436', 'c0fabc27', '94229ac3', 'dd34596f', '3e05f163', '8956a23f', 'cf31629e', '317f5077', 'beb6a151', '55b9e55e', 'cfe19a68']]
Skip doc_id a5042372 as it is too long to fit in the context.
Skip doc_id 8fc3eac8 as it is too long to fit in the context.
Adding content of doc 916dc60e to context.
Adding content of doc f69230a5 to context.
Skip doc_id 36409436 as it is too long to fit in the context.
Skip doc_id c0fabc27 as it is too long to fit in the context.
Skip doc_id 94229ac3 as it is too long to fit in the context.
Skip doc_id dd34596f as it is too long to fit in the context.
Skip doc_id 3e05f163 as it is too long to fit in the context.
Skip doc_id 8956a23f as it is too long to fit in the context.
Skip doc_id cf31629e as it is too long to fit in the context.
Skip doc_id 317f5077 as it is too long to fit in the context.
Skip doc_id beb6a151 as it is too long to fit in the context.
Skip doc_id 55b9e55e a